In [25]:
#Initialization for HateSpeechRecognition
import scipy as sp
import numpy as np
import pandas as pd
import sklearn
from sklearn import svm
import nltk
import re
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import zero_one_loss
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from scipy.stats import t
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mislavz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mislavz\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\mislavz\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [14]:
def nested_kfold_cv(clf, param_grid, X, y, k1=10, k2=3):
    
    err = []
    kfold = KFold(n_splits=k1, shuffle=True, random_state=42)
    
    # Outer loop
    for ind_train, ind_test in kfold.split(X):
        
        X_train, y_train, X_test, y_test = X[ind_train], y[ind_train], X[ind_test], y[ind_test]
        
        # Inner loop
        inn = GridSearchCV(clf,param_grid,n_jobs=5, cv=StratifiedKFold(n_splits=5, 
                                              random_state=42).split(X_train, y_train), 
                           verbose=2).fit(X_train, y_train)
        
        # Prediction based on the best selected params, the ones that minimize average error
        h = inn.best_estimator_.fit(X_train, y_train).predict(X_test)
        
        err.append(zero_one_loss(y_test, h))
        print(classification_report( y_test, h ))
    return err

In [15]:

def clean_text(rgx_list, text):
    new_text = text
    for rgx_match in rgx_list:
        new_text = re.sub(rgx_match, '', new_text)
    return new_text

In [16]:

stopwords_list = stopwords.words('english')
#other_exclusions = ["#ff", "ff", "rt","!",":","...",".","-","&","?"]
#stopwords_list.extend(other_exclusions)

dataset = pd.read_csv("dataset/labeled_data.csv")

tweets = dataset.tweet

ps = nltk.stem.PorterStemmer()

#ps.stem(dataset)

#Text cleaning and tokenization, then stemming then POS tagging
filtered_tweets=[];
tweet_tags = [];
filtered_tweets_stemmed=[];

for tweet in tweets:
    tweet = clean_text(["http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+",".*@.*:","&#*\w*","@[\w\-]+","[^\w\s]"],tweet)
    tweet = tweet.lower()
    word_tokens = word_tokenize(tweet)
    filtered_tweets.append([word for word in word_tokens if not word in stopwords_list])
    filtered_tweets_stemmed.append([ps.stem(word) for word in word_tokens if not word in stopwords_list])
    
for tweet in filtered_tweets:
    tweet_tags.append(nltk.pos_tag(tweet))

    

In [17]:
#Creating TfIdf vectorizer
ftss=[]
for tweet in filtered_tweets_stemmed:
    ftss.append(' '.join(tweet))

vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(ngram_range=(1,3),lowercase=False,max_features=1000,smooth_idf=False,norm=None,max_df=0.75,min_df=5)



In [18]:
X = vectorizer.fit_transform(ftss).toarray()
Y = dataset['class']

X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=42, test_size=0.3)



In [20]:
#SVM Classifier
C = [ 2e-2, 2e-1, 2e-0]
gama = [ 2e-2, 2e-1, 2e-0]
param = [{'svc__kernel': ['rbf'], 'svc__C': C}]
clf = svm.SVC(decision_function_shape='ovo')
scale = StandardScaler()
pipeline = Pipeline([('scaler', scale), ('svc', clf)])

svm_err = nested_kfold_cv(pipeline, param, X, Y)
#param_grid = [{}] 
#grid_search = GridSearchCV(pipeline, 
#                           param_grid,
#                          n_jobs = 5,
#                           cv=StratifiedKFold(n_splits=5, 
#                                              random_state=42).split(X_train, y_train), 
#                           verbose=2)
#model = grid_search.fit(X_train,y_train)
#y_pred = model.predict(X_test)
#report = classification_report( y_test, y_pred )
#print(report)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=5)]: Done  15 out of  15 | elapsed: 68.4min finished


             precision    recall  f1-score   support

          0       0.50      0.13      0.20       164
          1       0.86      0.96      0.91      1905
          2       0.80      0.61      0.69       410

avg / total       0.83      0.85      0.83      2479

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=5)]: Done  15 out of  15 | elapsed: 64.4min finished


             precision    recall  f1-score   support

          0       0.43      0.20      0.27       127
          1       0.89      0.96      0.92      1927
          2       0.83      0.64      0.72       425

avg / total       0.85      0.87      0.85      2479

Fitting 5 folds for each of 3 candidates, totalling 15 fits


...

             precision    recall  f1-score   support

          0       0.44      0.16      0.23       135
          1       0.87      0.96      0.92      1907
          2       0.85      0.65      0.74       436

avg / total       0.85      0.86      0.85      2478

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=5)]: Done  15 out of  15 | elapsed: 64.9min finished


             precision    recall  f1-score   support

          0       0.59      0.13      0.21       133
          1       0.89      0.97      0.93      1937
          2       0.84      0.67      0.75       408

avg / total       0.86      0.88      0.86      2478

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=5)]: Done  15 out of  15 | elapsed: 64.0min finished


             precision    recall  f1-score   support

          0       0.40      0.14      0.20       124
          1       0.90      0.96      0.93      1942
          2       0.82      0.72      0.77       412

avg / total       0.86      0.88      0.86      2478



In [19]:
#Logistic Regression

C = [2e-3, 2e-2, 2e-1, 2e-0, 2e-1, 2e-2, 2e-3]

std_scaler = StandardScaler()
clf = LogisticRegression(multi_class='ovr',solver='newton-cg')
pipeline = Pipeline([('scaler', std_scaler)  , ('clf', clf)])
param = [{'clf__C': C}]

logreg_err = nested_kfold_cv(pipeline, param, X, Y)



#TODO: logicstic regression, model selection, kfold

Fitting 5 folds for each of 7 candidates, totalling 35 fits


[Parallel(n_jobs=5)]: Done  35 out of  35 | elapsed:  7.0min finished


             precision    recall  f1-score   support

          0       0.49      0.19      0.27       164
          1       0.91      0.94      0.92      1905
          2       0.77      0.84      0.80       410

avg / total       0.86      0.87      0.86      2479

Fitting 5 folds for each of 7 candidates, totalling 35 fits


[Parallel(n_jobs=5)]: Done  35 out of  35 | elapsed:  6.5min finished


             precision    recall  f1-score   support

          0       0.44      0.31      0.36       127
          1       0.94      0.94      0.94      1927
          2       0.79      0.86      0.82       425

avg / total       0.89      0.89      0.89      2479

Fitting 5 folds for each of 7 candidates, totalling 35 fits


...

             precision    recall  f1-score   support

          0       0.40      0.23      0.29       135
          1       0.92      0.93      0.93      1907
          2       0.80      0.85      0.83       436

avg / total       0.87      0.88      0.87      2478

Fitting 5 folds for each of 7 candidates, totalling 35 fits


[Parallel(n_jobs=5)]: Done  35 out of  35 | elapsed:  6.0min finished


             precision    recall  f1-score   support

          0       0.57      0.26      0.36       133
          1       0.93      0.95      0.94      1937
          2       0.81      0.85      0.83       408

avg / total       0.89      0.90      0.89      2478

Fitting 5 folds for each of 7 candidates, totalling 35 fits


[Parallel(n_jobs=5)]: Done  35 out of  35 | elapsed:  6.7min finished


             precision    recall  f1-score   support

          0       0.39      0.25      0.30       124
          1       0.94      0.94      0.94      1942
          2       0.82      0.88      0.85       412

avg / total       0.89      0.90      0.89      2478



In [26]:
from scipy.stats.ttest_rel as paired_t_test
txt, t_stat, txt, p_val = paired_t_test(logreg_err, svm_err)

if p_val <= 0.05:
    print('\np-value = ', p_val, ', hipoteza se odbacuje.')
else:
    print('\np-value = ', p_val, ', hipoteza se ne odbacuje.')

ModuleNotFoundError: No module named 'scipy.stats.ttest_rel'